In [ ]:
import torch
import pickle
import pandas as pd
import numpy as np
import os
import re 
import random
import glob
import matplotlib.pyplot as plt
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader, random_split, Dataset
from torchvision.io import read_image
import pytorch_lightning as pl
import numpy as np

from torch.utils.data.sampler import SubsetRandomSampler
import torch.utils.data as tdata
from pathlib import Path

from PIL import Image
import collections

from torch.utils.data import Dataset
from torchvision import transforms as T
from sklearn.model_selection import GroupShuffleSplit
import pandas as pd
from statistics import mode

In [ ]:
#load the table

data = pd.read_csv("../imagine/image_quality.csv")

train_path = "../imagine/train.pickle"
test_path = '../imagine/test.pickle'
with open(train_path, 'rb') as f:
        x = pickle.load(f)

train_file = x.loc[x.class_label.isin(["psoriasis"]), :]
train_file.reset_index(inplace=True)
train_file = train_file.loc[:, ~train_file.columns.str.contains('^index')]


with open(test_path, 'rb') as f:
    x = pickle.load(f)

test_file = x.loc[x.class_label.isin(["psoriasis"]), :]
test_file.reset_index(inplace=True)
test_file = test_file.loc[:, ~test_file.columns.str.contains('^index')]
train_images = train_file.image_id.tolist()
train_images2 = [i[:-4] for i in train_images]
#print(test_images2)
train_meta_data = data.loc[data.platformImageId.isin(train_images2)]

test_images = test_file.image_id.tolist()
test_images2 = [i[:-4] for i in test_images]
#print(test_images2)
test_meta_data = data.loc[data.platformImageId.isin(test_images2)]
len(train_meta_data), len(test_meta_data)

# Checking that All the rows exist in the dataset

In [ ]:
data_dir = "imagine/images"
#create a list of the unique image ids that will iterately check their existance in the file
imgId_list = data['platformImageId'].unique().tolist()
print(len(imgId_list))
#a list of images that dont exist
nonExisting = []

for Id in imgId_list:
    path = data_dir+"/"+Id+".png"
    if Path(path).is_file():
        pass
    else:
        nonExisting.append(Id)
print(len(nonExisting))
print(len(data.loc[(data["platformImageId"].isin(nonExisting))]))
data = data.loc[(~data["platformImageId"].isin(nonExisting))] 
print(len(data))


In [ ]:
data_dir = "imagine/images"
blurry = data_dir+"/"+data.loc[data.blurry == True, "platformImageId"]+".png"
image = Image.open(blurry.iloc[0])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
far = data_dir+"/"+data.loc[data.too_far_away == True, "platformImageId"]+".png"
image = Image.open(far.iloc[1])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
light = data_dir+"/"+data.loc[data.bad_light == True, "platformImageId"]+".png"
image = Image.open(light.iloc[3])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
covered = data_dir+"/"+data.loc[data.lesion_covered == True, "platformImageId"]+".png"
image = Image.open(covered.iloc[2])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
resolution = data_dir+"/"+data.loc[data.low_resolution == True, "platformImageId"]+".png"
image = Image.open(resolution.iloc[0])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
framing = data_dir+"/"+data.loc[data.bad_framing == True, "platformImageId"]+".png"
image = Image.open(framing.iloc[2])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
SNV = data_dir+"/"+data.loc[data.diagnosis == "SNV", "platformImageId"]+".png"
image = Image.open(SNV.iloc[2])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
lesion = data_dir+"/"+data.loc[data.diagnosis == "L98.2", "platformImageId"]+".png"
image = Image.open(lesion.iloc[0])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))
plt.show()
CNS = data_dir+"/"+data.loc[data.diagnosis == "CNS", "platformImageId"]+".png"
image = Image.open(CNS.iloc[2])
image = T.ToTensor()(image)
plt.imshow(np.transpose(image, (1, 2, 0)))

plt.show()

In [ ]:
#print(data)
#find any row that has Nan values
data2 = data[data['diagnosis'].isna()]
print(len(data[data['diagnosis'].isna()]), len(data[data['platformImageId'].isna()]), len(data[data['dermId'].isna()]), len(data[data['patientId'].isna()]))


## delete CNV data as they are for this task trivial

In [ ]:
#SNV= skin not visible, CNS: clear skin, NEP = quality issues
#delete the CNV data
data = data[data['diagnosis'] != 'SNV']
print(len(data.dermId.unique())) #12 dermatologists in total
print(len(data))

# Ensuring no duplicates

In [ ]:
data = pd.read_csv("~/imaginePilot-v1.0/imagine/updated_image_quality.csv")
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
#data.head()
print(len(data))
#delete the duplicate images
data.drop_duplicates('platformImageId', inplace=True)
data.reset_index(inplace=True) 
print(len(data))
data = data.loc[:, ~data.columns.str.contains('^index')]
data

print(data)

# Data Analysis

In [ ]:
NEP_count = np.size(np.where(data["diagnosis"]== "NEP"))
data_count = data.shape
print(NEP_count)
print(data_count)
bad_framing_count = np.size(np.where(~data['bad_framing'].isnull()))
bad_light_count = np.size(np.where(~data['bad_light'].isnull()))
blurry_count = np.size(np.where(~data['blurry'].isnull()))
lesion_covered_count = np.size(np.where(~data['lesion_covered'].isnull()))
low_resolution_count = np.size(np.where(~data['low_resolution'].isnull()))
too_far_away_count = np.size(np.where(~data['too_far_away'].isnull()))

print("bad framing:", bad_framing_count, "bad_light:", bad_light_count, "blurry:", blurry_count, "lesion covered:", lesion_covered_count, "low resolution:", low_resolution_count, "too far away:", too_far_away_count)
x = np.array(["bad framing","bad light","blurry","lesion covered","low resolution","too far away"])
y = np.array([bad_framing_count, bad_light_count, blurry_count, lesion_covered_count, low_resolution_count, too_far_away_count])
plt.figure(figsize = (10, 5))
plt.bar(x,y)
for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center',
                 Bbox = dict(facecolor = 'yellow', alpha =.8))
plt.title("Distribution of Quality errors")
plt.xlabel("Error type")
plt.ylabel("number of images")
plt.savefig("errorTypeNumbers.png")     
plt.show()


# Patient count

In [ ]:
#how many patients in total
patient_count = len(train_file.patientId.unique())+len(test_file.patientId.unique())


In [ ]:
# todo: check how many patients are diagnosed different types of lesions
print("number of patients that are only diagnosed NEP: ", len(patient.loc[~patient["NEP"].isna() & patient["CNS"].isna() & patient["lesion"].isna()]))
print("number of patients that are only diagnosed CNS or lesion: ", len(patient.loc[patient["NEP"].isna()]))

# Split the data into common, NEP and CNS|lesion on patient level

In [ ]:
#CNS |& lesion: train section 1
# Note: a patient can have images of both cns and lesion, but each image is either lesion OR CNS
lesion_cns_patients = patient.loc[patient["NEP"].isna(),["patientId", "NEP","CNS","lesion"]] 
lesion_cns_index = data.patientId.isin(lesion_cns_patients.patientId.tolist()).values.tolist()
lesion_cns_images = data.iloc[lesion_cns_index]
print("lesion or CNS patients details: ", "#patients: ",len(lesion_cns_patients),"#images with CNS: ",lesion_cns_patients.CNS.sum(), "#images with lesion: ",lesion_cns_patients.lesion.sum(), "#images: ",len(lesion_cns_images))

#NEP only: test 1
NEP_patients = patient.loc[~patient["NEP"].isna() & patient["CNS"].isna() & patient["lesion"].isna(),["patientId", "NEP", "CNS","lesion"]]
NEP_index = data.patientId.isin(NEP_patients.patientId.tolist()).values.tolist()
NEP_images = data.iloc[NEP_index]
print("Only NEP patients details: ", "#patiens: ",len(NEP_patients),"#NEP images: ", NEP_patients.NEP.sum(), "NEP images: ", len(NEP_images))

#common
lesion_cns_nep_patients = patient.loc[~patient["NEP"].isna() & ~patient["CNS"].isna() & ~patient["lesion"].isna(),["patientId", "NEP","CNS","lesion"]]
lesion_cns_nep_index = data.patientId.isin(lesion_cns_nep_patients.patientId.tolist()).values.tolist()
lesion_cns_nep_images = data.iloc[lesion_cns_nep_index]
print("patients with a mix of all the labels: ", "#patients", len(lesion_cns_nep_patients), "# CNS images: ", lesion_cns_nep_patients.CNS.sum(), "# NEP images: ", lesion_cns_nep_patients.NEP.sum(),"# lesion images: ", lesion_cns_nep_patients.lesion.sum(), "#images", len(lesion_cns_nep_images))

lesion_nep_patients = patient.loc[~patient["NEP"].isna() & patient["CNS"].isna() & ~patient["lesion"].isna(),["patientId", "NEP","CNS","lesion"]]
lesion_nep_index = data.patientId.isin(lesion_nep_patients.patientId.tolist()).values.tolist()
lesion_nep_images = data.iloc[lesion_nep_index]
print("patients with a mix of lesion nep labels: ", "#patients: ",len(lesion_nep_patients),"# lesion images: ", lesion_nep_patients.lesion.sum(), "#NEP images: ", lesion_nep_patients.NEP.sum(), "# lesion nep images", len(lesion_nep_images))

cns_nep_patients = patient.loc[~patient["NEP"].isna() & ~patient["CNS"].isna() & patient["lesion"].isna(),["patientId", "NEP","CNS","lesion"]]
cns_nep_index = data.patientId.isin(cns_nep_patients.patientId.tolist()).values.tolist()
cns_nep_images = data.iloc[cns_nep_index]
print("patients with a mix of cns nep labels: ", "#patients: ", len(cns_nep_patients),"#cns images: ", cns_nep_patients.CNS.sum(), "#nep images: ", cns_nep_patients.NEP.sum(), "#images: ", len(cns_nep_images))


In [ ]:
common_labels = lesion_cns_nep_images.append(lesion_nep_images, ignore_index=True)
common_labels = common_labels.append(cns_nep_images, ignore_index=True)
print(len(common_labels.platformImageId.unique()),len(lesion_cns_nep_images)+len(lesion_nep_images)+len(cns_nep_images), len(cns_nep_patients)+len(lesion_nep_patients)+len(lesion_cns_nep_patients), len(common_labels.patientId.unique()))
common_labels 

In [ ]:
patient = data.patientId.unique()
counts = np.sum(patient, 0)
#plot classes
x = np.array(["CNS","NEP","other","psoriasis", "dermatitis", "acne","rosacea"])
y = np.array([counts.CNS, counts.NEP, counts.other, counts.psoriasis, counts.dermatitis, counts.acne, counts.rosacea])
plt.figure(figsize = (10, 5))
plt.bar(x,y)
for i in range(len(x)):
        plt.text(i, y[i], y[i], ha = 'center',
                 Bbox = dict(facecolor = 'yellow', alpha =.8))
plt.title("Distribution of Classes over total images")
plt.xlabel("Classes")
plt.ylabel("number of images")
plt.savefig("classdistribution.png")     
plt.show()

### check for superusers

In [ ]:
patient["count"] = np.sum(patient, 1)
#print(patient)
df = patient[patient["count"] > 15]
print(len(df))
#print(test)
superpatientsId = df.patientId #202 patients in total, 170 of these patients are in training. 
print(len(superpatientsId))
indices_superuserimages = np.where(test.patientId.isin(superpatientsId)) #4888 images for these patients on train dataset
superuser_images = test.iloc[indices_superuserimages]
print(len(superuser_images))

superuser_images = superuser_images.sort_values('patientId',ascending = False).groupby('patientId').head(15) #for each value of patientId I chose the first 15 images
print(superuser_images)


test= test.drop(test[(test.platformImageId.isin(superuser_images.platformImageId.tolist()).values.tolist()) & (test.patientId.isin(superpatientsId))].index)
print("test",test)

test.reset_index(inplace=True) 
test = test.loc[:, ~test.columns.str.contains('index')]


